In [2]:
%load_ext autoreload
%autoreload 2

In [50]:
from customerchurn.mlflow import MLFlowBase
from customerchurn.params import MLFLOW_URI
from customerchurn.params import EXPERIMENT_NAME

In [53]:
client = MLFlowBase( '[AUS] [MEL] [NLP709] CUSTOMERCHURN dummy', MLFLOW_URI)


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../raw_data/raw_en.csv')

df.info()

/tmp/ipykernel_8790/1251664015.py:6: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/raw_en.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457074 entries, 0 to 457073
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            457074 non-null  object 
 1   date            351179 non-null  object 
 2   recommendation  350678 non-null  object 
 3   review          457074 non-null  object 
 4   playing_hours   350678 non-null  float64
 5   source          457074 non-null  object 
 6   stars           106396 non-null  float64
 7   language        457074 non-null  object 
dtypes: float64(2), object(6)
memory usage: 27.9+ MB


In [4]:
df['language'].value_counts()

en    457074
Name: language, dtype: int64

In [5]:
df1 = df.head(10000).copy()
# creating small sample to train with

In [8]:
from customerchurn.pre_pipeline import create_pre_pipe
from customerchurn.pre_pipeline import series_prepro
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer



pipe = create_pre_pipe()

# pipe.fit(df1)

# df1['review_clean'] = pipe.transform(df1)[:,0]

In [9]:
df2 = df.head(1000).copy()
# creating small sample to train with

In [23]:
# !pip install -q -U tensorflow-text
# !pip install -q tf-models-official

You should consider upgrading via the '/home/cathal/.pyenv/versions/3.8.6/envs/customerchurn/bin/python -m pip install --upgrade pip' command.


In [10]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')


2022-02-10 13:58:37.980677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-10 13:58:37.980812: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/cathal/.pyenv/versions/3.8.6/envs/customerchurn/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the Ten

In [12]:
df2.recommendation.value_counts()

Recommended        673
Not Recommended    327
Name: recommendation, dtype: int64

In [13]:
df2.head()

,name,date,recommendation,review,playing_hours,source,stars,language
0,Expansion - Hearts of Iron IV: Man the Guns,2019-02-10,Recommended,&gt Played as German Reich&gt Declare war on B...,578.0,steam,NaN,en
1,Expansion - Hearts of Iron IV: Man the Guns,2019-02-07,Recommended,Very good game although a bit overpriced in my...,892.0,steam,NaN,en
2,Dead by Daylight,2018-06-14,Recommended,Out of all the reviews I wrote This one is pro...,676.0,steam,NaN,en
3,Dead by Daylight,2017-06-20,Recommended,Disclaimer I survivor main. I play games for f...,612.0,steam,NaN,en
4,Dead by Daylight,2016-12-12,Recommended,ENGLISH After playing for more than two years ...,2694.0,steam,NaN,en


In [14]:
X= df2['review']
y = df2['recommendation'].apply(lambda x: 0 if x == 'Not Recommended' else 1)

In [15]:
y.value_counts()

1    673
0    327
Name: recommendation, dtype: int64

In [ ]:
# AUTOTUNE = tf.data.AUTOTUNE
# batch_size = 32
# seed = 42

# raw_train_ds = tf.keras.utils.text_dataset_from_directory(
#     'customer/train',
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset='training',
#     seed=seed)

# class_names = raw_train_ds.class_names
# train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

# val_ds = tf.keras.utils.text_dataset_from_directory(
#     'customer/train',
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset='validation',
#     seed=seed)

# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# test_ds = tf.keras.utils.text_dataset_from_directory(
#     'customer/test',
#     batch_size=batch_size)

# test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [17]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess= 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [18]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

2022-02-10 13:58:57.414535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-10 13:58:57.414599: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-10 13:58:57.414621: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-1CG9GS9): /proc/driver/nvidia/version does not exist
2022-02-10 13:58:57.415964: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_mask', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [15]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76262903  0.9928099  -0.1861187   0.3667383   0.15233696  0.65504414
  0.9681154  -0.9486272   0.00216167 -0.9877732   0.06842678 -0.9763059 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946355  0.34321314  0.3323151  ...  0.2130079   0.7102079
  -0.05771136]
 [-0.28742105  0.31981066 -0.2301857  ...  0.58455014 -0.21329743
   0.7269204 ]
 [-0.6615694   0.6887674  -0.8743296  ...  0.10877217 -0.26173162
   0.4785526 ]
 ...
 [-0.22561109 -0.28925592 -0.07064441 ...  0.4756602   0.83277166
   0.40025324]
 [-0.29824194 -0.2747312  -0.05450527 ...  0.48849782  1.0955361
   0.18163359]
 [-0.4437828   0.00930762  0.07223727 ...  0.17290081  1.1833241
   0.07897999]]


In [20]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')

    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']

    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)

    return tf.keras.Model(text_input, net)

In [21]:
classifier_model = build_classifier_model()
# bert_raw_result = classifier_model(tf.constant(text_test))
# print(tf.sigmoid(bert_raw_result))

In [43]:
# loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# metrics = tf.metrics.BinaryAccuracy()

In [1]:
# epochs = 5
# steps_per_epoch = tf.data.experimental.cardinality(X).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

In [54]:
client.mlflow_create_run()

In [55]:
client.mlflow_log_param('optimizer', 'adam')

In [22]:
classifier_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

classifier_model.fit(X,y, batch_size =8 , epochs =1)

125/125 [==============================] - 53s 389ms/step - loss: 0.6358 - accuracy: 0.6550


In [62]:
client.mlflow_log_metric('accuracy', history1.history['accuracy'][0])

In [46]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [36]:
metrics =classifier_model.metrics

In [38]:
print(metrics.)

[<keras.metrics.Mean object at 0x7fd9c3a0d3a0>, <keras.metrics.MeanMetricWrapper object at 0x7fd9bc475670>]


In [32]:
history1 =classifier_model.history

In [33]:
history1.params

{'verbose': 1, 'epochs': 1, 'steps': 125}

In [60]:
history1.history['accuracy'][0]

0.6549999713897705

In [25]:
import multiprocessing
import time
import warnings
from tempfile import mkdtemp

# import category_encoders as ce
import joblib
import mlflow
# from TaxiFareModel.data import get_data, clean_df, DIST_ARGS
# from TaxiFareModel.encoders import TimeFeaturesEncoder, DistanceTransformer, AddGeohash, Direction, \
#     DistanceToCenter
# from TaxiFareModel.utils import compute_rmse, simple_time_tracker
from memoized_property import memoized_property
from mlflow.tracking import MlflowClient
from psutil import virtual_memory
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from termcolor import colored
# from xgboost import XGBRegressor

In [ ]:
MLFLOW_URI = "https://mlflow.lewagon.co/"
EXPERIMENT_NAME = "first_experiment"

In [ ]:
# create the model
import mlflow
import numpy as np
# from sklearn.linear_model import LogisticRegression
# from azureml.core import Workspace

# # connect to your workspace
# ws = Workspace.from_config()

# create experiment and start logging to a new run in the experiment
experiment_name = "azure-ml-in10-mins-tutorial"

# set up MLflow to track the metrics
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)
mlflow.autolog()

# set up the Logistic regression model
reg = 0.5
clf = classifier_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train the model
with mlflow.start_run() as run:
    clf.fit(X_train, y_train)

In [ ]:


MODEL_DIRECTY = "PipelineTest"  # must the same as PATH_TO_MODEL inside Makefile
MLFLOW_URI = "https://mlflow.lewagon.co/"


class Trainer(object):
    ESTIMATOR = "Linear"
    EXPERIMENT_NAME = "customerchurnMLFOWPOC"

    def __init__(self, X, y, **kwargs):
        """
        FYI:
        __init__ is called every time you instatiate Trainer
        Consider kwargs as a dict containig all possible parameters given to your constructor
        Example:
            TT = Trainer(nrows=1000, estimator="Linear")
               ==> kwargs = {"nrows": 1000,
                            "estimator": "Linear"}
        :param X:
        :param y:
        :param kwargs:
        """
        self.pipeline = None
        self.kwargs = kwargs
        self.local = kwargs.get("local", False)  # if True training is done locally
        self.mlflow = kwargs.get("mlflow", False)  # if True log info to nlflow
        self.experiment_name = kwargs.get("experiment_name", self.EXPERIMENT_NAME)  # cf doc above
        self.model_params = None  # for
        self.X_train = X
        self.y_train = y
        del X, y
        self.split = self.kwargs.get("split", True)  # cf doc above
        if self.split:
            self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(self.X_train, self.y_train,
                                                                                  test_size=0.15)
        self.nrows = self.X_train.shape[0]  # nb of rows to train on
        self.log_kwargs_params()
        self.log_machine_specs()

#     def get_estimator(self):
#         estimator = self.kwargs.get("estimator", self.ESTIMATOR)
#         if estimator == "Lasso":
#             model = Lasso()
#         elif estimator == "Ridge":
#             model = Ridge()
#         elif estimator == "Linear":
#             model = LinearRegression()
#         elif estimator == "GBM":
#             model = GradientBoostingRegressor()
#         elif estimator == "RandomForest":
#             model = RandomForestRegressor()
#             self.model_params = {  # 'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 200, num = 10)],
#                 'max_features': ['auto', 'sqrt']}
#             # 'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)]}
#         elif estimator == "xgboost":
#             model = XGBRegressor(objective='reg:squarederror', n_jobs=-1, max_depth=10, learning_rate=0.05,
#                                  gamma=3)
#             self.model_params = {'max_depth': range(10, 20, 2),
#                                  'n_estimators': range(60, 220, 40),
#                                  'learning_rate': [0.1, 0.01, 0.05]
#                                  }
#         else:
#             model = Lasso()
#         estimator_params = self.kwargs.get("estimator_params", {})
#         self.mlflow_log_param("estimator", estimator)
#         model.set_params(**estimator_params)
#         print(colored(model.__class__.__name__, "red"))
#         return model

#     def set_pipeline(self):
#         memory = self.kwargs.get("pipeline_memory", None)
#         dist = self.kwargs.get("distance_type", "euclidian")
#         feateng_steps = self.kwargs.get("feateng", ["distance", "time_features"])
#         if memory:
#              memory = mkdtemp()

#         # Define feature engineering pipeline blocks here
#         pipe_time_features = make_pipeline(TimeFeaturesEncoder(time_column='pickup_datetime'),
#                                            OneHotEncoder(handle_unknown='ignore'))
#         pipe_distance = make_pipeline(DistanceTransformer(distance_type=dist, **DIST_ARGS), StandardScaler())
#         pipe_geohash = make_pipeline(AddGeohash(), ce.HashingEncoder())
#         pipe_direction = make_pipeline(Direction(), StandardScaler())
#         pipe_distance_to_center = make_pipeline(DistanceToCenter(), StandardScaler())

#         # Define default feature engineering blocs
#         feateng_blocks = [
#             ('distance', pipe_distance, list(DIST_ARGS.values())),
#             ('time_features', pipe_time_features, ['pickup_datetime']),
#             ('geohash', pipe_geohash, list(DIST_ARGS.values())),
#             ('direction', pipe_direction, list(DIST_ARGS.values())),
#             ('distance_to_center', pipe_distance_to_center, list(DIST_ARGS.values())),
#         ]
#         # Filter out some bocks according to input parameters
#         for bloc in feateng_blocks:
#             if bloc[0] not in feateng_steps:
#                 feateng_blocks.remove(bloc)

#         features_encoder = ColumnTransformer(feateng_blocks, n_jobs=None, remainder="drop")

#         self.pipeline = Pipeline(steps=[
#                     ('features', features_encoder),
#                     ('rgs', self.get_estimator())],
#                                  memory=memory)



    def set_pipeline(self):
#         self.pipeline =classifier_model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
        self.pipeline = build_classifier_model()

# classifier_model.fit(X,y, batch_size =8 , epochs =1)
        

    @simple_time_tracker
    def train(self):
        tic = time.time()
        self.set_pipeline()
        self.pipeline.fit(self.X_train, self.y_train)
        # mlflow logs
        self.mlflow_log_metric("train_time", int(time.time() - tic))

    def evaluate(self):
        rmse_train = self.compute_rmse(self.X_train, self.y_train)
        self.mlflow_log_metric("rmse_train", rmse_train)
        if self.split:
            rmse_val = self.compute_rmse(self.X_val, self.y_val, show=True)
            self.mlflow_log_metric("rmse_val", rmse_val)
            print(colored("rmse train: {} || rmse val: {}".format(rmse_train, rmse_val), "blue"))
        else:
            print(colored("rmse train: {}".format(rmse_train), "blue"))

    def compute_rmse(self, X_test, y_test, show=False):
        if self.pipeline is None:
            raise ("Cannot evaluate an empty pipeline")
        y_pred = self.pipeline.predict(X_test)
        if show:
            res = pd.DataFrame(y_test)
            res["pred"] = y_pred
            print(colored(res.sample(5), "blue"))
        rmse = compute_rmse(y_pred, y_test)
        return round(rmse, 3)

    def save_model(self):
        """Save the model into a .joblib format"""
        joblib.dump(self.pipeline, 'model.joblib')
        print(colored("model.joblib saved locally", "green"))

    ### MLFlow methods
    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id)

    def mlflow_log_param(self, key, value):
        if self.mlflow:
            self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        if self.mlflow:
            self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)

    def log_estimator_params(self):
        reg = self.get_estimator()
        self.mlflow_log_param('estimator_name', reg.__class__.__name__)
        params = reg.get_params()
        for k, v in params.items():
            self.mlflow_log_param(k, v)

    def log_kwargs_params(self):
        if self.mlflow:
            for k, v in self.kwargs.items():
                self.mlflow_log_param(k, v)

    def log_machine_specs(self):
        cpus = multiprocessing.cpu_count()
        mem = virtual_memory()
        ram = int(mem.total / 1000000000)
        self.mlflow_log_param("ram", ram)
        self.mlflow_log_param("cpus", cpus)





In [ ]:
if __name__ == "__main__":
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Get and clean data
    experiment = "taxifare_set_YOURNAME"
    if "YOURNAME" in experiment:
        print(colored("Please define MlFlow experiment variable with your own name", "red"))
    params = dict(nrows=1000000,
                  local=False,  # set to False to get data from GCP (Storage or BigQuery)
                  optimize=True,
                  estimator="xgboost",
                  mlflow=True,  # set to True to log params to mlflow
                  experiment_name=experiment,
                  pipeline_memory=None,
                  distance_type="manhattan",
                  feateng=["distance_to_center", "direction", "distance", "time_features", "geohash"])
    print("############   Loading Data   ############")
    df = get_data(**params)
    df = clean_df(df)
    y_train = df["fare_amount"]
    X_train = df.drop("fare_amount", axis=1)
    del df
    print("shape: {}".format(X_train.shape))
    print("size: {} Mb".format(X_train.memory_usage().sum() / 1e6))
    # Train and save model, locally and
    t = Trainer(X=X_train, y=y_train, **params)
    del X_train, y_train
    print(colored("############  Training model   ############", "red"))
    t.train()
    print(colored("############  Evaluating model ############", "blue"))
    t.evaluate()
    print(colored("############   Saving model    ############", "green"))
    t.save_model()

In [19]:
!pip install pydot

You should consider upgrading via the '/home/cathal/.pyenv/versions/3.8.6/envs/customerchurn/bin/python -m pip install --upgrade pip' command.


In [21]:
# !sudo apt install graphviz

[sudo] password for cathal: 


In [24]:
tf.keras.utils.plot_model(classifier_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
